# **End of week 1 exercise**

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

## **1. Get a response from your favorite AI Tutor** 

In [1]:
import os
from openai import OpenAI
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display

In [2]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

API key looks good so far


In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

openai = OpenAI()

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [4]:
system_prompt = """You are the software engnieer, phd in mathematics, machine learning engnieer, and other topics"""
system_prompt += """
When responding, always use Markdown for formatting. For any code, use well-structured code blocks with syntax highlighting,
For instance:
```python

sample_list = [for i in range(10)]
```
Another example
```javascript
        function displayMessage() {
            alert("Hello, welcome to JavaScript!");
        }

```

Break down explanations into clear, numbered steps for better understanding. 
Highlight important terms using inline code formatting (e.g., `function_name`, `variable`).
Provide examples for any concepts and ensure all examples are concise, clear, and relevant.
Your goal is to create visually appealing, easy-to-read, and informative responses.

"""


In [5]:
def tutor_user_prompt(question):
    # Ensure the question is properly appended to the user prompt.
    user_prompt = (
        "Please carefully explain the following question in a step-by-step manner for clarity:\n\n"
    )
    user_prompt += question
    return user_prompt

In [6]:


def askTutor(question, MODEL):
    # Generate the user prompt dynamically.
    user_prompt = tutor_user_prompt(question)
    
    # OpenAI API call to generate response.
    if MODEL == 'gpt-4o-mini':
        print(f'You are getting response from {MODEL}')
        stream = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            stream=True
        )
    else:
        MODEL == 'llama3.2'
        print(f'You are getting response from {MODEL}')
        stream = ollama_via_openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            stream=True
        )

    # Initialize variables for response processing.
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    
    # Process the response stream and update display dynamically.
    for chunk in stream:
        # Safely access the content attribute.
        response_chunk = getattr(chunk.choices[0].delta, "content", "")
        if response_chunk:  # Check if response_chunk is not None or empty
            response += response_chunk
            # No replacement of Markdown formatting here!
            update_display(Markdown(response), display_id=display_handle.display_id)


In [7]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [8]:
askTutor(question=question, MODEL=MODEL_GPT)

You are getting response from gpt-4o-mini


Sure! Let's break down the code you provided step by step to understand its purpose and functionality.

### Code Explanation
The line of code in question is:
```python
yield from {book.get("author") for book in books if book.get("author")}
```

#### Step 1: Understanding the Context
- The line uses `yield from`, which is a Python statement used in a generator function. It allows the generator to yield all values from an iterable (like lists, sets, or other generators) effectively.

#### Step 2: Comprehension in Action
- The `{...}` syntax suggests that we are dealing with a **set comprehension**. Inside the curly braces is a comprehension that generates a set of authors from the `books`.

#### Step 3: The `for` Loop
- The expression `for book in books` iterates over an iterable called `books`. Here, `books` is expected to be a collection of dictionaries (i.e., a list of book dictionaries).

#### Step 4: Accessing Dictionary Values
- `book.get("author")` attempts to retrieve the value associated with the key `"author"` from each `book` dictionary. The `get` method is safe; it returns `None` if the key does not exist instead of raising a KeyError.

#### Step 5: Conditional Check
- The `if book.get("author")` part ensures that only books that have a valid author (not `None` or empty) will be included in the set.

#### Step 6: Creating a Set
- By enclosing the comprehension in `{}`, it generates a set of unique authors. If multiple books have the same author, that author will only appear once in the resulting set.

#### Step 7: Using `yield from`
- Finally, `yield from ...` allows the generator to yield each author in the created set one at a time.

### Summary
Putting all of this together, the entire line of code does the following:
1. Iterates through a collection of book dictionaries (`books`).
2. Extracts the `"author"` value for each book, checking that it exists.
3. Constructs a set of unique authors from those valid entries.
4. Yields each author back to the caller, making it possible to iterate over these authors in a calling context.

### Example
Consider a sample `books` list:

```python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C", "author": None},
    {"title": "Book D", "author": "Author 1"},
]
```

When we run the provided line, the output would yield:
- `Author 1`
- `Author 2`

This will ensure `Author 1` is only yielded once, despite appearing in multiple books. 

### Conclusion
This code snippet effectively filters out the unique authors from a collection of books and yields them, demonstrating the use of set comprehensions and generators in Python concisely.

## **2. Using both LLMs collaboratively approach**

- I thought about like similar the idea of a RAG (Retrieval-Augmented Generation) approach, is an excellent idea to improve responses by refining the user query and producing a polished, detailed final answer. Two LLM talking each other its cool!!! Here's how we can implement this:

**Updated Concept:**
1. Refine Query with Ollama:
    - Use Ollama to refine the raw user query into a well-structured prompt.
    - This is especially helpful when users input vague or poorly structured queries.
2. Generate Final Response with GPT:
    - Pass the refined prompt from Ollama to GPT to generate the final, detailed, and polished response.
3. Return the Combined Output:
    - Combine the input, refined query, and the final response into a single display to ensure clarity.

In [9]:
def refine_with_ollama(raw_question):
    """
    Use Ollama to refine the user's raw question into a well-structured prompt.
    """
    print("Refining the query using Ollama...")
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Refine and structure the following user input."},

        {"role": "user", "content": raw_question},
    ]
    response = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=messages,
        stream=False  # Non-streamed refinement
    )
    refined_query = response.choices[0].message.content
    return refined_query

In [10]:
def ask_with_ollama_and_gpt(raw_question):
    """
    Use Ollama to refine the user query and GPT to generate the final response.
    """
    # Step 1: Refine the query using Ollama
    refined_query = refine_with_ollama(raw_question)
    
    # Step 2: Generate final response with GPT
    print("Generating the final response using GPT...")
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": refined_query},
    ]
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True  # Stream response for dynamic display
    )

    # Step 3: Combine responses
    response = ""
    display_handle = display(Markdown(f"### Refined Query:\n\n{refined_query}\n\n---\n\n### Final Response:"), display_id=True)
    for chunk in stream:
        response_chunk = getattr(chunk.choices[0].delta, "content", "")
        if response_chunk:
            response += response_chunk
            update_display(Markdown(f"### Refined Query:\n\n{refined_query}\n\n---\n\n### Final Response:\n\n{response}"), display_id=display_handle.display_id)

In [11]:
# Example Usage
question = """
Please explain what this code does:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [12]:
ask_with_ollama_and_gpt(raw_question=question)

Refining the query using Ollama...
Generating the final response using GPT...


### Refined Query:

**Code Explanation**

```python
yield from {book.get("author") for book in books if book.get("author")}
```

This line of code is a Python expression that uses a combination of dictionary lookups and generator expressions. Its purpose is to extract the "author" values from a list of dictionaries (`books`).

Here's a breakdown:

1. `for book in books`: This part iterates over each dictionary in the `books` collection.

2. `if book.get("author")`: For each book dictionary, it checks if the value associated with the key `"author"` exists using the `.get()` method. If the key does not exist or its corresponding value is `None`, the book will be skipped and moved on to the next one.

3. `{book.get("author") for book in books}` : This part uses a dictionary comprehension, similar to those used to create dictionaries, but instead of producing a single dictionary value based on some key-value pairs, it generates a collection (in this case an iterable) which contains the values associated with the `"author"` key.

4. `yield from ...`: Once we have the list of authors (`{book.get("author") for book in books if book.get("author")}`), we feed them into another function that uses the `yield from` statement within its definition. Here, `yield from` is used to return values yielded by an iterable instead of returning an iterable itself.

However, it seems like a more optimal approach should be implemented as follows:

```python
def get_authors(books):
    for book in books:
        if "author" in book and book["author"]:
            yield book["author"]
```

Or with `dict.get()` :

```python
def get_authors(books):
    for book in books:
        yield book.get("author")
```

In this context, `yield from` doesn't make sense because you're yielding up the individual items returned by the dictionary lookups and not directly from another iterable of authors.

This code snippet is used when working with collections of dictionaries, where you need to loop over each item to get its "author" value.

---

### Final Response:

Let's break down the provided code and explanation step-by-step to clarify its functionality and discuss potential improvements. 

### Original Line of Code

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Breakdown of the Code

1. **Iteration over `books`**:
   - `for book in books`: This part loops through each `book` in the `books` collection.

2. **Conditional Author Check**:
   - `if book.get("author")`: This checks whether the `"author"` key exists in the `book` dictionary and ensures its value is not `None` or an empty string. The `.get()` method is used here to avoid a `KeyError` if the key does not exist.

3. **Set Comprehension**:
   - `{book.get("author") for book in books if book.get("author")}`: This creates a *set* of unique authors. By using a set comprehension, you ensure that each author is included only once.

4. **Yielding Authors**:
   - `yield from ...`: This part is intended to yield each item from the iterable generated by the preceding set comprehension. However, the use of `yield from` is not optimal in this context.

### Suggested Improvements

The original use of `yield from` combined with a set comprehension can lead to confusion and inefficiency. Here are two alternative implementations:

#### Using a Generator Function

```python
def get_authors(books):
    for book in books:
        if "author" in book and book["author"]:
            yield book["author"]
```

### Explanation of the Generator Function

1. **Function Definition**:
   - `def get_authors(books)`: Defines a function that takes a list of dictionaries (`books`).

2. **Iterating through the Collection**:
   - The `for book in books` loop goes through each book.

3. **Checking for Author**:
   - Condition checks if the key `"author"` exists and contains a valid value (non-empty).

4. **Yielding Each Author**:
   - `yield book["author"]`: Returns each valid author's name as the loop iterates. 

#### Alternatively, Using `dict.get()`

```python
def get_authors(books):
    for book in books:
        author = book.get("author")
        if author:  # This will check if author is not None or an empty string.
            yield author
```

### Explanation of the Alternative Implementation

1. **Using `.get()` for Safety**:
   - `author = book.get("author")`: This safely retrieves the `"author"` value.

2. **Conditional Check**:
   - The `if author:` statement allows you to directly check if the author variable contains a valid value before yielding.

### Conclusion

- The original expression using `yield from` is not ideal for yielding authors directly from the comprehension set; it is designed for yielding from another generator or iterable.
- The suggested implementations provide clearer and more Pythonic solutions while efficiently handling the extraction of authors from a collection of dictionaries. 

### Example Usage

Assuming you have a collection of books defined like this:
```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Author B"},
    {"title": "Book 4"},  # No author
]
```

You could retrieve authors by calling:
```python
for author in get_authors(books):
    print(author)
```

This would produce:
```
Author A
Author B
``` 

These revisions reflect cleaner, more efficient, and Pythonic code that directly addresses the intended goal.